In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI
from pdf2image import convert_from_path
import base64
import io
import os

# Replace with your OpenAI API key
#openai.api_key = 'your-api-key'

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=<ADD_YOUR_API_KEY>,
)


frmt=open('icon_format_last.txt','r').read()

# Step 1: Take input of a PDF path
data_path = input("Enter the path to the folder: ")
pdf_path = data_path + '/pdfs'
op_path = data_path + '/op'
opjson_path = data_path + '/op_JSON'

for ip_pdf in os.listdir(pdf_path):
    if not ip_pdf.endswith(".pdf"):
        continue
    op_file = os.path.join(op_path, f"{os.path.splitext(ip_pdf)[0]}.txt")
    opjson_file = os.path.join(opjson_path, f"{os.path.splitext(ip_pdf)[0]}.json")

    #print(os.path.join(pdf_path,ip_pdf))
    #print(op_file)
    # Step 2: Convert PDF pages to images in memory
    images = convert_from_path(os.path.join(pdf_path,ip_pdf))
    # Initialize the conversation context
    messages = [
        {"role": "system", "content": "You are an AI model that can extract the meaning of images."},
        {"role": "user", "content": "Extract the key-values of the page-wise images of a PDF (genomic report). I will give the images (pages) soon one by one."}
    ]
    meaning=''

    # Step 3: Send the initial prompt
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        max_tokens= 4000,
        temperature= 0.2
    )
    
    # Extract and print the initial response from the assistant
    print(response.choices[0].message.content)

    # Step 4: Iteratively process each page image and send it to OpenAI
    for i, image in enumerate(images):
        # Convert the image to bytes in memory
        img_byte_arr = io.BytesIO()
        image.save(img_byte_arr, format='JPEG')
        img_byte_arr = img_byte_arr.getvalue()
    
        # Encode the image to base64 directly from memory
        encoded_string = base64.b64encode(img_byte_arr).decode('utf-8')
        
        # Send a message including the encoded image
        #messages.append({"role": "user", "content": f"This is page {i + 1} of the PDF: {encoded_string}"})
        messages.append({"role": "user", 
        "content": [
                {
                    "type": "text",
                    "text": f"This is page {i + 1} of the PDF: "
                },
                {
                    "type": "image_url",
                    "image_url": {
                    "url": f"data:image/jpeg;base64,{encoded_string}"
                    }
                }
                ]})
    
        # Send the updated conversation to OpenAI
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            max_tokens= 4000,
            temperature= 0.2
        )
        
        
        # Extract and print the assistant's response
        assistant_reply = response.choices[0].message.content
        print(f"Meaning extracted for Page {i + 1}: {assistant_reply}")
        meaning= meaning + assistant_reply
        
        # Append the assistant's response to the message history
        messages.append({"role": "assistant", "content": assistant_reply})
        
    
      

    messages2 = [
        {"role": "system", "content": "You are an AI model that can understand information extracted from a Genomic report."},
        {"role": "user", "content": "Read the genomic report summary and structure the output according to the specified format given below."}
    ]
    messages2.append({"role": "user", 
        "content":[
                {
                    "type": "text",
                    "text": "Summary of the Genomic report: " + meaning
                },
                {
                    "type": "text",
                    "text": "The response format: " + frmt
                }
    ]})

    response2 = client.chat.completions.create(
            model="gpt-4o",
            messages=messages2,
            max_tokens= 4000,
            temperature= 0.2
        )
    
    open(op_file, 'w').write(response2.choices[0].message.content)
    oplist= response2.choices[0].message.content
    messages3 = [
        {"role": "system", "content": "You are an AI model that can convert a hierarchical list of key:value to a JSON file."},
        {"role": "user", "content": "Hierarchical list: " + oplist}
    ]
    
    response3 = client.chat.completions.create(
            model="gpt-4o",
            messages=messages3,
            max_tokens= 4000,
            temperature= 0.2
        )
    open(opjson_file, 'w').write(response3.choices[0].message.content)
    

In [ ]:
    messages3 = [
        {"role": "system", "content": "You are an AI model that can convert a hierarchical list of key:value to a JSON file."},
        {"role": "user", "content": "Read the genomic report summary and structure the output according to the specified format given below."}
    ]
    messages2.append({"role": "user", 
        "content":
                {
                    "type": "text",
                    "text": "Hierarchical list: " + response2.choices[0].message.content
                }
    })

In [ ]:
import json
import re

def parse_hierarchical_list(file_path):
    data = {}
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            
            # Ignore lines starting with '###' or numerical labels
            if line.startswith('###'):
                continue
            
            # Use regex to capture key-value pairs and strip leading labels like '1.1', '1.2'
            match = re.match(r'\d+\.\d+ (.*?): (.*)', line)
            if match:
                key = match.group(1).strip()
                value = match.group(2).strip()
                data[key] = value

    return data

def save_as_json(data, json_file_path):
    with open(json_file_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)

# Input file path
input_file = ''

# Parse the hierarchical list to dictionary
parsed_data = parse_hierarchical_list(input_file)

# Output JSON file path
output_file = 'doc2_1.json'

# Save the parsed data as JSON
save_as_json(parsed_data, output_file)

print(f"Hierarchical list has been converted to JSON and saved as '{output_file}'")
